# 知的システム_Webサイトエキスパートシステム
##### 更新日：2023/07/24 7:25
##### 最終編集者：高橋


### 記事を参照（スクレイピング）

##### csvからリストを作成し、スクレイピングを実行

In [1]:
%%capture
!pip install pandas

In [4]:
import pandas as pd
# CSVファイルのパスを指定します。適宜、ファイルのパスを変更してください。
file_path = '/content/ニュース一覧_エンタメ.csv'
# CSVファイルをPandasのDataFrameに読み込みます。
df = pd.read_csv(file_path,encoding='shift_jis')
urls = df.iloc[:, 1].tolist()

In [5]:
import requests , os
from bs4 import BeautifulSoup

# URLから 記事タイトル と 記事の内容をスクレイピング
def import_news(url):
  # requestsを使って、ウェブページのHTMLを取得する
  response = requests.get(url)
  html = response.text
  # BeautifulSoupを使って、HTMLを解析する
  soup = BeautifulSoup(html, 'html.parser')

  # 必要な情報を抽出する
  article_title = soup.find('h1',{'class': 'sc-epOimh'}).text
  article_content = soup.find('div', {'class': 'article_body'}).text

  return article_title,article_content

In [6]:
# スクレイピング実行
directory = 'news'
os.makedirs(directory, exist_ok=True)

for url in urls:
  try:
    title, content = import_news(url)
    file = open(f"./news/{title}.txt", "w") # 保存先パス
    file.write(content)
    file.close()
  except:
    pass

##### タイトルとURLを紐づけ(やらなくてOK)

In [ ]:
urls_dic = {
    # 'タイトル' , 'URL'
    '旧統一教会所有の土地、隣接する国士舘「撤退を」 \u3000東京・多摩':'https://news.yahoo.co.jp/articles/0b53218a811768500ba0632fdec574b7cad68b12', # 旧統一教会所有の土地、隣接する国士舘「撤退を」
    '北朝鮮「核兵器の使用条件に該当」\u3000米戦略原潜の韓国寄港を非難':'https://news.yahoo.co.jp/articles/ab0e02b37308d1c2e4b22a491d92e641b5020700', # 北朝鮮「核兵器の使用条件に該当」　米戦略原潜の韓国寄港を非難
    '「不適切な内容」ビッグモーター、社長のメディア批判ＬＩＮＥを謝罪':'https://news.yahoo.co.jp/articles/3092daf89980572cdc032a4a42402a7d10045964', # 「不適切な内容」ビッグモーター、社長のメディア批判ＬＩＮＥを謝罪
    '東海道新幹線、車内チャイムに別れ\u3000聞き納め乗車も':'https://news.yahoo.co.jp/articles/f2b4de2449fa5444e4c6363093d8b637ad6bafb8', # 東海道新幹線、車内チャイムに別れ　聞き納め乗車も
    '＜ルポ路上売春＞歌舞伎町で「立ちんぼ」3年\u3000ネットカフェ暮らし、ホスト通いの末に':'https://news.yahoo.co.jp/articles/a6a50f861b540ea5a89508c1ad95e2605cd641c5', # ＜ルポ路上売春＞歌舞伎町で「立ちんぼ」3年　ネットカフェ暮らし、ホスト通いの末に
}

### 形態素解析

##### ライブラリをインストール

In [7]:
%%capture
# Mecab
!pip install mecab-python3
# ipadic（辞書）
!pip install ipadic

##### 実行

In [8]:
import MeCab
import ipadic
# ipa辞書を使う

def extract_nouns(text):
    mecab = MeCab.Tagger(ipadic.MECAB_ARGS)  # 単語に分割するためのTaggerを作成
    parsed_text = mecab.parse(text)  # 文章を形態素解析する

    nouns = []
    for word in parsed_text.split():
        word_parsed = mecab.parse(word).split('\t')[0]  # 単語を形態素解析して基本形を取得
        part_of_speech = mecab.parse(word).split('\t')[1].split(',')[0]  # 単語の品詞を取得
        if part_of_speech == '名詞':
            nouns.append(word_parsed)  # 名詞のみリストに追加

    tango = []
    for noun in nouns:
      if noun not in ['名詞','動詞','助詞','形容詞','助動詞','記号',r'[0-9]+']:
        tango.append(noun)
    return tango

In [9]:
def create_prule(folder_path):
  prule = [] # 形態素解析用

  for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) and file_path.endswith(".txt"):
      with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
        name = file_path.split('/')[-1].split('.')[0]

        #input(text)
        #形態素解析
        nouns = extract_nouns(text)
        prule.append([name]) # タイトル
        # prule.append([urls_dic[name]]) # URL
        for word in nouns:
          prule[-1].append(word)
  return prule
# 実行
folder_path = "news"  # 対象のフォルダのパスを指定
prule = create_prule(folder_path)

In [10]:
# 確認
for i in prule:
  print(i)

['「ザ・マミィ」酒井貴士、一般女性と７月７日「七夕婚」\u3000３年前にマッチングアプリで出会う', 'ザ・マミィ', '酒井', '貴士', 'お笑い', 'コンビ', 'ザ・マミィ', '酒井', '貴士', '３', '２', '１', '９', '日', '放送', 'ＴＢＳ', '系', '水曜日', 'ダウン', 'タウン', '水曜', '後', '１', '０', '時', '一般', '女性', '結婚', '発表', '連体詞', '日', '特別', '企画', '８', '人', '芸人', '中', 'ホント', '結婚', '一', '人', '今夜', '誰', '結婚', '発表', '告知', '公式', 'ツイッター', 'ダウン', '浜田', '雅', '松本', '人志', '様子', '公開', '酒井', '接頭', '相手', '出会い', '３', '年', '前', 'マッチングアプリ', '最初', 'マッチ', '方', '副詞', '副詞', '感じ', '初対面', '接続詞', '接頭', '交際', '１', '年', 'キングオブコント', '決勝', '交際', '条件', '２', '１', '年', 'キングオブコント', '準', '接頭', '優勝', 'ザ・マミィ', '酒井', '公私', 'とも', '幸せ', '接続詞', '結婚', '決め手', 'ケンカ', '４', '回', '５', '回', '副詞', '大', '接頭', 'ゲンカ', '一生', '相手', 'ラスト', '１', '回', '再会', '決め手', '仲直り', '上京', '同棲', '現在', '状況', '番組', '結婚', '発表', '後', '相方', '林田', '洋平', '先輩', '岡野', '陽一', '証人', 'サイン', '酒井', '感動', '声', '接頭', '２', '人', 'サイン', '接頭', '２', '人', '僕', '連体詞', '大粒', '涙', '晴れ', '７月', '７', '日', '午後', '７', '時', '婚姻', '届', '提出', '様子', '放送', '連体詞', '日', '特別', '企画', '８', '

### エキスパートシステム

In [11]:
import os
import random

##### システム

In [16]:
def answer(inputline, prule):
  no = 0
  for singlep in prule:
    if rulematch(inputline, singlep):
      no+=1
  if no==0:
    print(f"{botname}: どうぞ、続けてください")
  else:
    limit = random.randint(0,no-1)
    no = 0
    for singlep in prule:
      if rulematch(inputline, singlep):
        if no==limit:
          print(f"{botname}: {singlep[0]}")
          break
        no+=1
# enddef

In [59]:
def rulematch(inputline, singlep):
  count = 0
  for i in range(len(singlep)-1):
    if singlep[i] == "":
      count+=1
    elif singlep[i] in inputline:
      count+=1
    else:
      count+=0 # 何もしない
  if count >= 24: #24文字以上で検索
    return True
  else:
    return False
# end

##### 実行

In [ ]:
# data4
# プロダクションルール
botname="さくら"

#["おはよう","","","","おはようございます。どうしました？"],
prule = []

for url in urls:
  prule.append([url])
  for word in nouns:
    prule[urls.index(url)].append(word)

print(prule)

[['https://news.yahoo.co.jp/articles/0b53218a811768500ba0632fdec574b7cad68b12', 'ビッグモーター', '２', '０', '日', '午後', '埼玉', '県', '所沢', '市', '中古', '車', '販売', '大手', 'ビッグモーター', '東京', '兼重', '宏行', '社長', '保険', '金', '不正', '請求', '問題', '報道', '全店', '店長', '宛て', 'メディア', '批判', '文書', 'ＬＩＮＥ', '送信', 'こと', '同社', '広報', '担当', '者', '２', '０', '日', '夜', '産経新聞', '取材', '回答', '形', '文章', '当社', '兼重', '社長', '従業', '員', '向け', 'もの', '事実', '関係', 'LINE', '文書', 'ビッグモーター', '社長', '全', '接頭', '店長', 'LINE', 'メディア', '批判', '報道', '文書', '今回', '不正', '請求', '問題', '報道', '世間', '関心', 'ため', '不満', '内容', '広報', '担当', '者', '産経新聞', '前半', '部分', '不適切', '内容', 'こと', '猛省', 'おり', '兼重', '氏', 'メディア', '批判', '謝罪', '内容', '回答', '寄せ', '兼重', '氏', '１', '９', '日', '全', '接頭', '店長', 'ＬＩＮＥ', '送信', 'メディア', '批判', '文書', '産経新聞', 'ビッグモーター', '電話', 'メール', '問い合わせ', 'EOS']]


<p>[質問リスト]</p>
<ul>
<li>B’zが提供した「DIGNITY」の歌詞を担当したメンバーは誰で、作曲を担当したメンバーは誰ですか？</li>
<li>影山 優佳さんのラストステージの記事を知りたい</li>
<li>高田 純次さんの「じゅん散歩」に関する記事を知りたい</li>
</ul>

In [48]:
botname = 'さくら'
# エキスパートシステム実行部
print(f"{botname}: さて、どうしました？")
try:
  while True:
    inputline = input("あなた: ")

    if inputline == 'end': # 'end'を入力する事で終了
      print('終了します。')
      break

    # 回答へ
    answer(inputline, prule)
    print() #改行
except EOFError:
  print(f"{botname}: 終わりましょう。")

さくら: さて、どうしました？
あなた: B’zが提供した「DIGNITY」の歌詞を担当したメンバーは誰で、作曲を担当したメンバーは誰ですか？
さくら: Ado歌唱×B’z楽曲提供　映画『沈黙の艦隊』主題歌で強力タッグ　稲葉浩志「時代を象徴する歌声」と称賛

あなた: B’zが提供した「DIGNITY」の歌詞を担当したメンバーは誰で、作曲を担当したメンバーは誰ですか？
さくら: Ado歌唱×B’z楽曲提供　映画『沈黙の艦隊』主題歌で強力タッグ　稲葉浩志「時代を象徴する歌声」と称賛

あなた: 影山 優佳さんのラストステージの記事を知りたい
さくら: 「やっぱり寂しい」日向坂46影山優佳、卒業セレモニーで大粒の涙　悔しさにじませながらも未来への決意「私が人生をもって証明する」

あなた: 影山 優佳さんのラストステージの記事を知りたい
さくら: 「やっぱり寂しい」日向坂46影山優佳、卒業セレモニーで大粒の涙　悔しさにじませながらも未来への決意「私が人生をもって証明する」

あなた: 高田 純次さんの「じゅん散歩」に関する記事を知りたい
さくら: 高田純次さんの「テキトー」に迫る　散歩中のギャグににじむ気遣い

あなた: 高田 純次さんの「じゅん散歩」に関する記事を知りたい
さくら: 高田純次さんの「テキトー」に迫る　散歩中のギャグににじむ気遣い

あなた: end
終了します。
